In [1]:
"""
================================================================================
NOTEBOOK 01: ENVIRONMENT SETUP, VERIFICATION
================================================================================
Copyright (c) 2026 Ajay Mahale. All Rights Reserved.
License: CC BY-NC-ND 4.0
https://creativecommons.org/licenses/by-nc-nd/4.0/


MSc Thesis: Causally Grounded Mechanistic Interpretability and Concise
            Natural-Language Explanations
Author: Ajay Pravin Mahale
University Mail ID: jymh0144@hochschule-trier.de
Personal Mail ID: Mahale.ajay01@gmail.com
Institution: Hochschule Trier
Supervisor: Prof. Dr. Ernst Georg Haffner

Purpose: Verify environment setup and confirm GPT-2 performs IOI task correctly
Expected Runtime: ~2 minutes
================================================================================
"""

# ==============================================================================
# CELL 1: Install Dependencies
# ==============================================================================
# TransformerLens: Library for mechanistic interpretability
# NumPy 1.26.4: Version compatible with TransformerLens

!pip install -q transformer-lens
!pip install -q numpy==1.26.4

print("✅ Dependencies installed")

✅ Dependencies installed


In [2]:
# ==============================================================================
# CELL 2: Verify GPU Availability
# ==============================================================================

import torch
import numpy as np
import random
import sys
from datetime import datetime

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"✅ Seed set: {SEED}")

print("=" * 60)
print("ENVIRONMENT & REPRODUCIBILITY")
print("=" * 60)
print(f"Random Seed:     {SEED}")
print(f"Python:          {sys.version.split()[0]}")
print(f"PyTorch:         {torch.__version__}")
print(f"CUDA Available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version:    {torch.version.cuda}")
    print(f"GPU:             {torch.cuda.get_device_name(0)}")
try:
    import transformer_lens
    print(f"TransformerLens: {transformer_lens.__version__}")
except:
    pass
print(f"Timestamp:       {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)


!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")


✅ Seed set: 42
ENVIRONMENT & REPRODUCIBILITY
Random Seed:     42
Python:          3.12.12
PyTorch:         2.9.0+cu126
CUDA Available:  True
CUDA Version:    12.6
GPU:             Tesla T4
Timestamp:       2026-02-06 12:45:43
Fri Feb  6 12:45:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A 

In [3]:
# ==============================================================================
# CELL 3: Load GPT-2 Small Model
# ==============================================================================
# GPT-2 Small: 12 layers, 12 heads per layer, 124M parameters
# This is the model used throughout the thesis

from transformer_lens import HookedTransformer
import torch
print("Loading GPT-2 Small...")
model = HookedTransformer.from_pretrained("gpt2-small")

print(f"\n✅ Model loaded successfully!")
print(f"   Device: {model.cfg.device}")
print(f"   Layers: {model.cfg.n_layers}")
print(f"   Heads per layer: {model.cfg.n_heads}")
print(f"   Parameters: 124M")

Loading GPT-2 Small...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer

✅ Model loaded successfully!
   Device: cuda
   Layers: 12
   Heads per layer: 12
   Parameters: 124M


In [4]:
# ==============================================================================
# CELL 4: Test IOI Task (Critical Verification)
# ==============================================================================
# The Indirect Object Identification (IOI) task:
# Given: "When Mary and John went to the store, John gave a drink to"
# Expected: Model predicts "Mary" (the indirect object)
# This is the foundation of the entire thesis.


import torch
prompt = "When Mary and John went to the store, John gave a drink to"

# Tokenize and run forward pass
tokens = model.to_tokens(prompt)
logits = model(tokens)

print(f"Input prompt: \"{prompt}\"")
print(f"Token count: {tokens.shape[1]}")
print(f"Vocabulary size: {logits.shape[-1]}")

# Get prediction
last_token_logits = logits[0, -1, :]
top_token = torch.argmax(last_token_logits).item()
predicted_word = model.to_string([top_token])

print(f"\n🎯 Model's prediction: '{predicted_word}'")

Input prompt: "When Mary and John went to the store, John gave a drink to"
Token count: 15
Vocabulary size: 50257

🎯 Model's prediction: ' Mary'


In [5]:
# ==============================================================================
# CELL 5: Verification Assertion
# ==============================================================================
# Clean the prediction (remove leading space if present)

predicted_clean = predicted_word.strip().lower()

if predicted_clean == "mary":
    print("\n" + "="*60)
    print("✅ VERIFICATION PASSED")
    print("="*60)
    print("GPT-2 correctly identifies Mary as the indirect object.")
    print("Environment is ready for thesis experiments.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("❌ VERIFICATION FAILED")
    print("="*60)
    print(f"Expected: 'Mary', Got: '{predicted_word}'")
    print("DO NOT PROCEED. Check model loading and dependencies.")
    print("="*60)



✅ VERIFICATION PASSED
GPT-2 correctly identifies Mary as the indirect object.
Environment is ready for thesis experiments.


In [6]:
# ==============================================================================
# CELL 6: Additional IOI Examples (Additional Verification)
# ==============================================================================
# Tested multiple IOI prompts to ensure robust performance

import torch
test_prompts = [
    ("When Mary and John went to the store, John gave a drink to", "Mary"),
    ("When Alice and Bob went to the park, Bob handed a flower to", "Alice"),
    ("When Sarah and Tom visited the museum, Tom showed a painting to", "Sarah"),
]

print("\n" + "="*60)
print("ADDITIONAL IOI VERIFICATION")
print("="*60)

all_passed = True
for prompt, expected in test_prompts:
    tokens = model.to_tokens(prompt)
    logits = model(tokens)
    pred_token = torch.argmax(logits[0, -1, :]).item()
    predicted = model.to_string([pred_token]).strip()

    status = "✅" if predicted.lower() == expected.lower() else "❌"
    if predicted.lower() != expected.lower():
        all_passed = False

    print(f"{status} Expected: {expected}, Got: {predicted}")

if all_passed:
    print("\n✅ All IOI tests passed! Ready to proceed.")
else:
    print("\n⚠️ Some tests failed. Investigate before proceeding.")




ADDITIONAL IOI VERIFICATION
✅ Expected: Mary, Got: Mary
✅ Expected: Alice, Got: Alice
✅ Expected: Sarah, Got: Sarah

✅ All IOI tests passed! Ready to proceed.
